# Getting Started with Llama Stack !

This notebook will walk you throught the steps to get started on LlamaStack
The first few steps need to happen outside of this notebook to get a stack server running.
Please look at this [guide](https://github.com/meta-llama/llama-stack/blob/main/docs/getting_started.md) for detailed instructions. 

For more client examples for other apis ( agents, memory, safety ) in llama_stack please refer to the [llama-stack-apps](https://github.com/meta-llama/llama-stack-apps/tree/main/examples).

In this notebook, we will showcase a few things to help you get started,
- Start the Llama Stack Server 
- How to use simple text and vision inference llama_stack_client APIs

### Starting the Llama Stack Server 

1. Get Docker container
```
$ docker login
$ docker pull llamastack/distribution-meta-reference-gpu
```

2. pip install the llama stack client package 
For this purpose, we will directly work with pre-built docker containers and use the python SDK
```
$ git clone https://github.com/meta-llama/llama-stack-apps.git
$ cd llama-stack-apps
$ yes | conda create -n stack-test python=3.10 
$ conda activate stack-test
$ pip install llama_stack llama_stack_client
```
This will install `llama_stack` and `llama_stack_client` packages. 
This will enable you to use the `llama` cli. 

3. Download model 
```
$ llama download --help 
$ llama download --source meta --model-id Llama3.2-11B-Vision-Instruct --meta-url <META_URL>
```

4. Configure the Stack Server
```
For GPU inference, you need to set these environment variables for specifying local directory containing your model checkpoints, and enable GPU inference to start running docker container.
$ export LLAMA_CHECKPOINT_DIR=~/.llama
```

5.  Run the Stack Server
```
$ llama stack run local-gpu --port 5000
```

The server has started correctly if you see outputs like the following 
```
...
...
Listening on :::5000
INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://[::]:5000 (Press CTRL+C to quit)
```

### Llama Stack Client examples

In [5]:
from llama_stack_client import LlamaStackClient

In [6]:
host = "localhost"
port = 5001
client = LlamaStackClient(base_url=f"http://{host}:{port}")

In [7]:
# For this notebook we will be working with the latest Llama3.2 vision models
model = "Llama3.2-1B-Instruct"

### Inference APIs ( chat_completion ) 

In [8]:
# Simple text example
iterator = client.inference.chat_completion(
    model=model,
    messages=[
        {
            "role": "user",
            "content": "Write a haiku on llamas"
        }
    ],
    stream=True
)

for chunk in iterator:
    print(chunk.event.delta, end="", flush=True)

TypeError: Missing required arguments; Expected either ('messages' and 'model_id') or ('messages', 'model_id' and 'stream') arguments to be given

### Multimodal Inference 

In [ ]:
import base64
import mimetypes

from PIL import Image

# We define a simple utility function to take a local image and
# convert it to as base64 encoded data url
# that can be passed to the server.
def data_url_from_image(file_path):
    mime_type, _ = mimetypes.guess_type(file_path)
    if mime_type is None:
        raise ValueError("Could not determine MIME type of the file")

    with open(file_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")

    data_url = f"data:{mime_type};base64,{encoded_string}"
    return data_url

with open("dog.jpg", "rb") as f:
    img = Image.open(f).convert("RGB")

img.show()


In [ ]:
# we can reuse the same chat_completion interface for multimodal inference too
# Use path to local file
data_url = data_url_from_image("dog.jpg")
iterator = client.inference.chat_completion(
    model=model,
    messages=[
        {
            "role": "user",
            "content": [
                { "image": { "uri": data_url } },
                "Write a haiku describing the image"
            ]
        }
    ],
    stream=True
)

for chunk in iterator:
    print(chunk.event.delta, end="", flush=True)